In [17]:
from adam.casadi import KinDynComputations
import casadi as cs
from math import sqrt
import numpy as np

In [18]:
model_path = "/Users/tommasoandina/Documents/Doosan_h2515-1/h2515.blue.urdf" 
joints_name_list = 'joint1', 'joint2', 'joint3', 'joint4', 'joint5', 'joint6' 
root_link = 'base'

kinDyn = KinDynComputations(model_path, joints_name_list, root_link)
num_dof = kinDyn.NDoF

In [19]:
H = cs.SX.sym('H', 4, 4)
# The joint values
s = cs.SX.sym('s', num_dof)
# The base velocity
v_b = cs.SX.sym('v_b', 6)
# The joints velocity
s_dot = cs.SX.sym('s_dot', num_dof)
# The base acceleration
v_b_dot = cs.SX.sym('v_b_dot', 6)
# The joints acceleration
s_ddot = cs.SX.sym('s_ddot', num_dof)

# initialize
mass_matrix_fun = kinDyn.mass_matrix_fun()
coriolis_term_fun = kinDyn.coriolis_term_fun()
gravity_term_fun =  kinDyn.gravity_term_fun()
bias_force_fun = kinDyn.bias_force_fun()
Jacobian_fun = kinDyn.jacobian_fun("link6")


In [20]:
M = kinDyn.mass_matrix_fun()
#M = M[:6, :6]
print('Mass matrix:\n', cs.DM(M(H, s)))

Mass matrix:
 
[[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.00235887]]


In [16]:
h = kinDyn.bias_force_fun()
print('Bias force term:\n', cs.DM(h(H, s, v_b, s_dot)))

Bias force term:
 [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
